# Apert综合征与正常人脸二分类非端到端模型

In [1]:
from deepface import DeepFace
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import math
import joblib
from sklearn.metrics import classification_report

设置数据源路径以及保存模型位置

In [2]:
NORMALIZED_IMG_DIR = './dataset/'
clf_model_name = 'Apert-normal.pkl'
pca_model_name = 'Apert-normal_pca.pkl'

读取所有遗传病类型

In [3]:
class_list = os.listdir(NORMALIZED_IMG_DIR)

引入FaceNet

In [4]:
model = DeepFace.build_model('Facenet')

使用FaceNet获取人脸embedding特征向量

In [5]:
embeddings = {}

In [6]:
for class_name in class_list:
#     embeddings[class_name] = DeepFace.represent(os.listdir(os.path.join(NORMALIZED_IMG_DIR, class_name)), model_name = 'Facenet', enforce_detection=False)
    embeddings[class_name] = []
    for file_name in os.listdir(os.path.join(NORMALIZED_IMG_DIR, class_name)):
        print(f"getting {file_name}'s embedding")
        embeddings[class_name].append(
            DeepFace.represent(os.path.join(NORMALIZED_IMG_DIR, class_name, file_name), model_name = 'Facenet', model=model, enforce_detection=False)
        )

getting 1.jpg's embedding
getting 10.jpg's embedding
getting 101.jpg's embedding
getting 102.jpg's embedding
getting 103.jpg's embedding
getting 104.jpg's embedding
getting 105.jpg's embedding
getting 106.jpg's embedding
getting 107.jpg's embedding
getting 108.jpg's embedding
getting 109.jpg's embedding
getting 11.jpg's embedding
getting 110.jpg's embedding
getting 111.jpg's embedding
getting 112.jpg's embedding
getting 114.jpg's embedding
getting 115.jpg's embedding
getting 116.jpg's embedding
getting 117.jpg's embedding
getting 118.jpg's embedding
getting 119.jpg's embedding
getting 12.jpg's embedding
getting 120.jpg's embedding
getting 121.jpg's embedding
getting 122.jpg's embedding
getting 123.jpg's embedding
getting 124.jpg's embedding
getting 125.jpg's embedding
getting 126.jpg's embedding
getting 127.jpg's embedding
getting 128.jpg's embedding
getting 129.jpg's embedding
getting 13.jpg's embedding
getting 130.jpg's embedding
getting 132.jpg's embedding
getting 134.jpg's embeddin

In [7]:
X = []
Y = []
for key, value in embeddings.items():
    X = X + value
    Y = Y + [key] * len(value)

In [13]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1)

网格搜索

In [14]:
best_score = -1
best_kernel = ''
best_pca_dim = -1
min_dim = math.inf
for class_name in class_list:
    min_dim = min(min_dim, len(os.listdir(os.path.join(NORMALIZED_IMG_DIR, class_name))))
min_dim = min(min_dim, 129)
print(f'searching pca dim in range 1 t4o {min_dim}')
for n_dim in range(1, min_dim):
    pca = PCA(n_components=n_dim)
    pca = pca.fit(X)
    X_dr = pca.transform(Xtrain)
    kernels = ["linear","poly","rbf","sigmoid"]
    for kernel in kernels:
        clf = SVC(kernel=kernel)
        score = cross_val_score(clf, X_dr, Ytrain, cv=7, scoring='accuracy').mean()
        if score >= best_score:
            best_score = score
            best_pca_dim = n_dim
            best_kernel = kernel
            print("The accuracy under kernel %s and pca dimension %d is %f" % (kernel, n_dim, score))

searching pca dim in range 1 t4o 129
The accuracy under kernel linear and pca dimension 1 is 0.839009
The accuracy under kernel linear and pca dimension 2 is 0.849543
The accuracy under kernel rbf and pca dimension 2 is 0.849543
The accuracy under kernel linear and pca dimension 3 is 0.849543
The accuracy under kernel sigmoid and pca dimension 3 is 0.852189
The accuracy under kernel rbf and pca dimension 10 is 0.854834
The accuracy under kernel linear and pca dimension 12 is 0.857480
The accuracy under kernel rbf and pca dimension 12 is 0.857480
The accuracy under kernel sigmoid and pca dimension 12 is 0.857480
The accuracy under kernel sigmoid and pca dimension 13 is 0.857480
The accuracy under kernel sigmoid and pca dimension 15 is 0.857480
The accuracy under kernel sigmoid and pca dimension 16 is 0.857480
The accuracy under kernel sigmoid and pca dimension 22 is 0.857480
The accuracy under kernel rbf and pca dimension 25 is 0.857480
The accuracy under kernel sigmoid and pca dimensio

In [15]:
pca = PCA(n_components=best_pca_dim)
pca = pca.fit(X)
X_dr = pca.transform(X)
joblib.dump(pca, pca_model_name)

['Apert-normal_pca.pkl']

In [16]:
clf = SVC(kernel=best_kernel, probability = True)
X_train_dr = pca.transform(Xtrain)
X_test_dr = pca.transform(Xtest)
clf.fit(X_train_dr, Ytrain)
Y_test_predict = clf.predict(X_test_dr)
target_names = class_list
print(classification_report(Ytest, Y_test_predict, target_names=target_names))

              precision    recall  f1-score   support

       Apert       0.83      1.00      0.91        20
      normal       1.00      0.83      0.90        23

    accuracy                           0.91        43
   macro avg       0.92      0.91      0.91        43
weighted avg       0.92      0.91      0.91        43



In [12]:
joblib.dump(clf, clf_model_name)

['Apert-normal.pkl']